In [1]:
import os
import lucene

from java.io import File 
from org.apache.lucene.analysis.standard import StandardAnalyzer 
from org.apache.lucene.document import Document, Field 
from org.apache.lucene.index import IndexWriter, IndexWriterConfig 
from org.apache.lucene.store import SimpleFSDirectory 
from org.apache.lucene.util import Version
from org.apache.lucene.search import IndexSearcher 
from org.apache.lucene.index import IndexReader 
from org.apache.lucene.queryparser.classic import QueryParser 

from org.apache.lucene import document, store, util
import numpy as np
import csv

import gensim
import time

import nltk
from nltk.corpus import stopwords

import numpy as np


Using gpu device 0: GeForce GTX 960M (CNMeM is enabled with initial size: 50.0% of memory, cuDNN not available)


In [2]:
# PATHS 
luceneIndexPath = '/home/tarun/PE/lucene/luceneIndexDirectory/'
corpus = '/home/tarun/PE/newCorpus/'
trainingFilePath = '/home/tarun/PE/Dataset/training_set.tsv'

lucene.initVM()

# ANALYZER
analyzer = StandardAnalyzer(util.Version.LUCENE_CURRENT) 

# DIRECTORY
directory = SimpleFSDirectory(File(luceneIndexPath))


# INDEX WRITER
writerConfig = IndexWriterConfig(util.Version.LUCENE_CURRENT, analyzer) 
writer = IndexWriter(directory, writerConfig)

print writer.numDocs()
# INDEXING ALL DOCUMENTS/ARTICLES IN THE CORPUS
for fileName in os.listdir(corpus):
    document = Document()
    article = os.path.join(corpus, fileName)
    content = open(article, 'r').read()
    document.add(Field("text", content, Field.Store.YES, Field.Index.ANALYZED))
    writer.addDocument(document)
print writer.numDocs()
writer.close()

# INDEX READER
reader = IndexReader.open(directory)
searcher = IndexSearcher(reader)

# QUERYING FOR A QUESTION
queryParser = QueryParser(util.Version.LUCENE_CURRENT, "text", analyzer)


0
23917


In [3]:
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()


In [4]:
sentences = MySentences('/home/tarun/PE/newCorpus/') # a memory-friendly iterator
model = gensim.models.Word2Vec(sentences, workers=8) # min_count=5, size=100,
model.save('word2vecModel')


In [5]:
model = gensim.models.Word2Vec.load('word2vecModel')

failed = 0

def getSimilarity(sentence1, sentence2):
    global failed
    tokens1 = nltk.word_tokenize(sentence1)
    tokens2 = nltk.word_tokenize(sentence2)
    try:
        tokens1 = filter(lambda x: x in model.vocab, tokens1)
        tokens2 = filter(lambda x: x in model.vocab, tokens2)
        nonStopWords1 = [word for word in tokens1 if word not in stopwords.words('english')]
        nonStopWords2 = [word for word in tokens2 if word not in stopwords.words('english')]
        wordSimScores = []
        for word1 in nonStopWords1:
            for word2 in nonStopWords2:
                wordSimScores.append(model.similarity(word1, word2))
        if (len(wordSimScores) == 0):
            return 0
        return np.mean(wordSimScores)
    except:
        failed+=1
    return 0
print failed

0


In [ ]:
#DocText with options
import pickle
with open('/home/tarun/PE/Dataset/final_test_set.pkl', 'rb') as f:
    test = pickle.load(f)

listOfHits = [1,2,3,4,5,6]

for eachNum in listOfHits:
    print eachNum
    submissionFileName = "luceneModelTest" + str(eachNum) + ".csv"
    answers = ['A', 'B', 'C', 'D']
    submissionFile = open(submissionFileName, "w")
    writer = csv.writer(submissionFile, delimiter=',')
    writer.writerow(['id', 'correctAnswer'])
    checkRows = []
    num = 0
    tic = time.time()
    for row in test:
        if(num % 1000 == 0):
            print num, time.time() - tic
        num = num + 1
        question = row[1]
        answerScores = []
        query = queryParser.parse(queryParser.escape(question))
        hits = searcher.search(query, eachNum)
        docsScores = [hit.score for hit in hits.scoreDocs]
        docTexts = ""
        for hit in hits.scoreDocs:
            doc_id = hit.doc
            #print doc_id, hit.toString()
            docT = searcher.doc(hit.doc)
            docText = docT.get("text").encode("utf-8")
            docTexts = docTexts + docText

        for option in [row[2], row[3], row[4], row[5]]:
            # escape for handling special characters like "/"
            
            similarity = getSimilarity(docTexts, option) #docTexts with options
            answerScores.append(similarity)
            if similarity == 0:
                checkRows.append(test.index(row))

        writer.writerow([row[0], answers[answerScores.index(np.max(answerScores))]])

1
0 1.21593475342e-05


In [ ]:
#DocText with options and question with options... weight of both (sum)
import pickle
with open('/home/tarun/PE/Dataset/final_test_set.pkl', 'rb') as f:
    test = pickle.load(f)

listOfHits = [1,2,3,4,5,6]

for eachNum in listOfHits:
    print eachNum
    submissionFileName = "luceneModelTest" + str(eachNum) + ".csv"
    answers = ['A', 'B', 'C', 'D']
    submissionFile = open(submissionFileName, "w")
    writer = csv.writer(submissionFile, delimiter=',')
    writer.writerow(['id', 'correctAnswer'])
    checkRows = []
    for row in test:
        question = row[1]
        answerScores = []
        for option in [row[2], row[3], row[4], row[5]]:
            # escape for handling special characters like "/"
            query = queryParser.parse(queryParser.escape(question))
            hits = searcher.search(query, eachNum)
            docsScores = [hit.score for hit in hits.scoreDocs]
            docTexts = ""
            for hit in hits.scoreDocs:
                doc_id = hit.doc
                #print doc_id, hit.toString()
                docT = searcher.doc(hit.doc)
                docText = docT.get("text").encode("utf-8")
                docTexts = docTexts + docText
            
            similarity1 = getSimilarity(docTexts, option) #docTexts with options
            similarity2 = getSimilarity(question, option)
            similarity = similarity1 + similarity2
            answerScores.append(similarity)
            if similarity == 0:
                checkRows.append(test.index(row))

        writer.writerow([row[0], answers[answerScores.index(np.max(answerScores))]])